In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras 
import os
import cv2

In [2]:
path1=r"D:\seatbelt_Noseatbelt"
cate=['with_seatbelt','without_seatbelt']

In [3]:
for i in cate:
    folders =os.path.join(path1, i)
    print(folders)

D:\seatbelt_Noseatbelt\with_seatbelt
D:\seatbelt_Noseatbelt\without_seatbelt


In [4]:
for i in cate:
    folders = os.path.join(path1, i)
    for image in os.listdir(folders):
        image_path =os.path.join(folders, image)


In [5]:
image_size = 200
path1=r"D:\seatbelt_Noseatbelt"
cate=['with_seatbelt','without_seatbelt']
input_image =[]
for i in cate:
    folders = os.path.join(path1, i)
    label =cate.index(i)
    #j=0
    for image in os.listdir(folders):
        image_path =os.path.join(folders, image)
        image_array = cv2.imread(image_path)
        #print(image_path)
        #print(j)
        #j=j+1
        image_array = cv2.resize(image_array, (image_size, image_size))
        input_image.append([image_array, label])

In [6]:
np.random.shuffle(input_image)

In [7]:
X=[]
Y=[]
for X_values, labels in input_image:
    X.append(X_values)
    Y.append(labels)

In [8]:
X =np.array(X)
Y =np.array(Y)

In [9]:
len(X)

20999

In [10]:
X.shape

(20999, 200, 200, 3)

In [11]:
11660+9339 

20999

In [12]:
20999-12000

8999

In [13]:
8999+4000

12999

In [14]:
20999*.8

16799.2

In [15]:
x_train = X[0:5999]  
x_test = X[5999:9999]

y_train = Y[0:5999]   
y_test = Y[5999:9999] 

x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [16]:
#X =X/255

In [17]:
x_train=x_train/255  # Now in range [0, 1] Pixel intensity values fall within the range [0.0, 1.0].
x_test=x_test/255

In [19]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

# Load MobileNetV2 as the base model (without top layers)
base_model = MobileNetV2(input_shape=X.shape[1:], include_top=False, weights='imagenet')

# Freeze base model weights (so pretrained features remain unchanged)
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Replaces Flatten() for better efficiency
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)  # Regularization
output = Dense(2, activation='softmax')(x)  # 2 classes: Seatbelt / No Seatbelt

# Define the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
#model.summary()


C:\Users\Kalyani Patil\AppData\Local\Temp\ipykernel_3144\1216720578.py:7: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=X.shape[1:], include_top=False, weights='imagenet')


In [20]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.001) 

In [21]:
model.compile(optimizer =adam, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [22]:
x_train.shape

(5999, 200, 200, 3)

In [23]:
y_train.shape

(5999,)

In [24]:
model.fit(x_train, y_train, epochs=15,validation_split=.2,batch_size=32)

Epoch 1/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 96s 554ms/step - accuracy: 0.7509 - loss: 0.5033 - val_accuracy: 0.8775 - val_loss: 0.2794
Epoch 2/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 79s 529ms/step - accuracy: 0.8895 - loss: 0.2475 - val_accuracy: 0.9108 - val_loss: 0.2084
Epoch 3/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 83s 553ms/step - accuracy: 0.9292 - loss: 0.1796 - val_accuracy: 0.9242 - val_loss: 0.1760
Epoch 4/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 81s 538ms/step - accuracy: 0.9461 - loss: 0.1392 - val_accuracy: 0.9300 - val_loss: 0.1685
Epoch 5/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 81s 527ms/step - accuracy: 0.9588 - loss: 0.1096 - val_accuracy: 0.9425 - val_loss: 0.1524
Epoch 6/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 84s 539ms/step - accuracy: 0.9619 - loss: 0.0978 - val_accuracy: 0.9508 - val_loss: 0.1471
Epoch 7/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 81s 538ms/step - accuracy: 0.9718 - loss: 0.0748 - val_accuracy: 0.9233 - val_loss: 0.1933
Epoch 8/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 81s 543ms/step - accuracy: 0.9739 - loss: 0

In [25]:
pred = model.predict(x_test)
pred_classes = pred.argmax(axis=1)

from sklearn.metrics import confusion_matrix, classification_report
tab = confusion_matrix(y_test, pred_classes)
print('Confusion Matrix:')
print(tab)
print('------------------------------------------------------')
print('Classification Report:')
print(classification_report(y_test, pred_classes))

125/125 ━━━━━━━━━━━━━━━━━━━━ 52s 403ms/step
Confusion Matrix:
[[2135  121]
 [  78 1666]]
------------------------------------------------------
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.95      0.96      2256
           1       0.93      0.96      0.94      1744

    accuracy                           0.95      4000
   macro avg       0.95      0.95      0.95      4000
weighted avg       0.95      0.95      0.95      4000



In [26]:
model.save("seatbelt_noseatbelt_model.h5")